In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils import parse_UK_Data, generate_large_distance_matrix


dataset = "../datasets/UK10_1.txt"
meta_data, distance_data, station_data = parse_UK_Data(dataset)
station_coordinates = pd.read_csv('../datasets/UK10_1_coordinates.csv', encoding='utf-8')
points_coordinate = station_coordinates[["lat", "long"]].to_numpy()

api = "Tfmt6Sxyn-nfinSHUyUo3xmVDWdtwAUyh6C8Y3HTZhU"

route = [[0,1,2,3], [4,5,6,7,8,9]]

In [2]:
meta_data["F-C Empty (l/100km)"] = 12.5
meta_data["F-C Full (l/100km)"] = 15
meta_data

,Customer Amount,Vehicle Curb Weight(kg),Max Load(kg),Minimum Speed(km/h),Maximum Speed(km/h),F-C Empty (l/100km),F-C Full (l/100km)
0,10,6350,3650,20,90,12.5,15


In [3]:
distance_matrix_df, travel_time_matrix, error_matrix_df = generate_large_distance_matrix(points_coordinate, api)

In [48]:
def get_results(vehicles:list, distance_matrix:pd.DataFrame, demand_data:pd.DataFrame, meta_data:pd.DataFrame, travel_time_matrix:pd.DataFrame) -> pd.DataFrame:
    
    def _get_total_distance(vehicles:list, distance_matrix:pd.DataFrame) -> list:
        total_vehicle_distance = []
        for vehicle, vehicle_route in enumerate(vehicles):
            distance = 0
            for i in range(len(vehicle_route) - 1):
                distance += distance_matrix.iloc[vehicle_route[i]][vehicle_route[i+1]]
            total_vehicle_distance.append(distance/1e3)
        return total_vehicle_distance
    
    def _get_total_load(vehicles:list, demand_data:pd.DataFrame) -> list:
        total_vehicle_load = []
        for vehicle, vehicle_route in enumerate(vehicles):
            load = 0
            for i in range(len(vehicle_route) - 1):
                load += int(demand_data.iloc[vehicle_route[i]]["Demand(kg)"])
            total_vehicle_load.append(load)
        return total_vehicle_load
    
    def _get_estimated_fuel_consumption(vehicles:list, meta_data:pd.DataFrame, distance_matrix:pd.DataFrame) -> list:
        total_vehicle_fuel_consumption = []
        for vehicle, vehicle_route in enumerate(vehicles):
            fc = 0
            for i in range(len(vehicle_route) - 1):
                #Distance in 100km
                distance = distance_matrix.iloc[vehicle_route[i]][vehicle_route[i+1]]/1e5
                #Demand in kg
                load = int(demand_data.iloc[vehicle_route[i]]["Demand(kg)"])
                #Fuel consumption between nodes driving empty vehicle
                fuel_consumption_empty = distance * meta_data['F-C Empty (l/100km)']
                load_rate = load / float(meta_data['Max Load(kg)'])
                #Additional fuel consumption when adding load at from_index
                fuel_consumption_load = distance * load_rate * (meta_data['F-C Full (l/100km)'] - meta_data['F-C Empty (l/100km)'])
                fc += np.float(fuel_consumption_empty + fuel_consumption_load)
                
            total_vehicle_fuel_consumption.append(fc)
    
        return total_vehicle_fuel_consumption
        
    def _get_avg_estimated_fuel_conspumtion(vehicle_distances:list, vehicle_fc:list) -> list:
        return  [fc/(dist/10) for dist,fc in zip(vehicle_distances, vehicle_fc)]
    
    def _format_time(vehicle_times:list) -> str:
        times = []
        for vehicle in vehicle_times:
            hours, rem = divmod(vehicle, 3600)
            minutes, seconds = divmod(rem, 60)
            if(minutes + hours == 0):
                times.append("{:05.2f}s".format(seconds))
            elif(minutes > 0 and hours == 0):
                times.append("{:0>2}:{:05.2f}".format(int(minutes),seconds))
            else:
                times.append("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
        return times
    
    def _get_total_travel_time(vehicles:list, travel_time_matrix:pd.DataFrame) -> list:
        total_vehicle_travel_time = []
        for vehicle, vehicle_route in enumerate(vehicles):
            travel_time = 0
            for i in range(len(vehicle_route) - 1):
                travel_time += travel_time_matrix.iloc[vehicle_route[i]][vehicle_route[i+1]]
            total_vehicle_travel_time.append(travel_time)
        return total_vehicle_travel_time
    
    def _get_avg_speed(distances:list, travel_times:list) -> list:
        return [dist/(seconds/60/60) for dist, seconds in zip(distances, travel_times)]
    
    vehicle_distances = _get_total_distance(vehicles, distance_matrix)
    vehicle_loads = _get_total_load(vehicles, demand_data)
    vehicle_fc = _get_estimated_fuel_consumption(vehicles, meta_data, distance_matrix)
    vehicle_avg_fc = _get_avg_estimated_fuel_conspumtion(vehicle_distances, vehicle_fc)
    vehicle_total_travel_time = _get_total_travel_time(vehicles, travel_time_matrix)
    vehicle_avg_speed = _get_avg_speed(vehicle_distances, vehicle_total_travel_time)
    
    results = pd.DataFrame()
    results["Total distance (km)"] = np.array(vehicle_distances)
    results["Total load (kg)"] = np.array(vehicle_loads)
    results["Total Estimated Fuel Consumption (L)"] = vehicle_fc
    results["Avg Estimated Fuel Conspumtion (L/100km)"] = vehicle_avg_fc
    results["Avg Speed (km/h)"] = vehicle_avg_speed  
    results["Total Travel Time (s)"] = vehicle_total_travel_time
    results["Travel Time hh:mm:ss"] = _format_time(vehicle_total_travel_time)
    
    
    return results

In [49]:
get_results(route, distance_matrix_df, station_data, meta_data, travel_time_matrix)

,Total distance (km),Total load (kg),Total Estimated Fuel Consumption (L),Avg Estimated Fuel Conspumtion (L/100km),Avg Speed (km/h),Total Travel Time (s),Travel Time hh:mm:ss
0,352.340,1535,45.650579,1.295640,74.820032,16953,04:42:33.00
1,951.047,2322,122.276894,1.285708,74.102747,46203,12:50:03.00
